In [1]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb

max_features = 10000  # number of words to consider as features
max_len = 500  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [2]:
x_train[:5]

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       [ 687,   23,    4, ...,   21,   64, 2574],
       [   0,    0,    0, ...,    7,   61,  113]], dtype=int32)

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
____________________________________________

In [8]:
import os
import numpy as np

data_dir = '../../deep-learning-with-python-notebooks-master/'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()

f.close()

lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]

float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values
    
mean = float_data[:200000].mean(axis=0)
float_data -= mean
std = float_data[:200000].std(axis=0)
float_data /= std

def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]
        yield samples, targets
        
lookback = 1440
step = 6
delay = 144
batch_size = 128

train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(float_data,
                    lookback=lookback,
                    delay=delay,
                    min_index=200001,
                    max_index=300000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(float_data,
                     lookback=lookback,
                     delay=delay,
                     min_index=300001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# This is how many steps to draw from `val_gen`
# in order to see the whole validation set:
val_steps = (300000 - 200001 - lookback) // batch_size

# This is how many steps to draw from `test_gen`
# in order to see the whole test set:
test_steps = (len(float_data) - 300001 - lookback) // batch_size

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
            input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 32)          5152      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, None, 32)          5152      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [12]:
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
                        input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))

model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, None, 32)          5152      
_________________________________________________________________
gru (GRU)                    (None, 32)                6336      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 13,793
Trainable params: 13,793
Non-trainable params: 0
_________________________________________________________________


In [14]:
def maxArea(nums):
    seen = set()
    def area(r,c):
        if not (0<=r<len(nums) and 0<=c<len(nums[0])) and (r,c) not in seen\
            and nnums[r][c]:
            return 0
        seen.add((r,c))
        return (1+area(r+1,c)+area(r-1,c)+area(r,c-1)+area(r,c+1))
    return max(area(r,c) for r in range(len(nums)) for c in range(len(nums[0])))

In [15]:
def maxArea(nums):
    seen = set()
    def area(r,c):
        if not (0<=r<len(nums) and 0<=c<len(nums[0])
               and (r,c) not in seen and nums[r][c]):
            return 0
        seen.add((r,c))
        return (1+area(r+1,c) + area(r-1,c)+area(r,c-1)+area(r,c+1))
    return max(area(r,c) for r in range(len(nums)) for c in range(len(nums[0])))

In [16]:
pwd

'/Users/dxm/Documents/HM_DXM/ML_Code/src'

In [23]:
import pandas as pd
df_data = pd.read_csv("../../../../Desktop/test.csv", header=None, sep='\t')

In [27]:
df_data

,0,1,2,3,4,5,6,7,8,9,...,49,50,51,52,53,54,55,56,57,58
0,3924465689,0.000000,0.000000,0.00,0.00,32.135135,5.032291,0.797297,0.404757,36.864865,...,NaN,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
1,807474631,0.014815,0.005979,0.10,0.04,31.701550,6.988260,0.773902,0.707960,37.739974,...,1.0,46.512821,NaN,1733.076923,NaN,1.0,11183.0,17.0,0.998569,46.512821
2,2483444575,0.007149,0.006717,0.08,0.08,32.355705,5.525472,0.815063,0.462098,38.125373,...,0.0,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,1.444444
3,3521518453,0.051282,0.000000,0.06,0.00,28.760000,6.089918,0.856000,0.352503,35.824000,...,NaN,0.200000,NaN,7.400000,NaN,0.0,148.0,1.0,0.000000,0.200000
4,2186210797,0.000000,0.020642,0.00,0.09,32.350951,6.263924,0.828753,0.480838,37.019068,...,0.0,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
5,1488677087,0.010689,0.002268,0.09,0.02,33.336066,6.254579,0.667008,0.490711,38.350823,...,NaN,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
6,1506575294,0.010518,0.003744,0.88,0.32,33.282168,6.450054,0.788212,0.430971,38.574070,...,0.0,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
7,2992512629,0.011925,0.004477,0.21,0.08,33.208333,7.168749,0.698390,0.489044,38.259716,...,NaN,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
8,5410816,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.000000,NaN,0.000000,NaN,0.0,0.0,0.0,0.000000,0.000000
9,1561404066,0.024012,0.010568,1.67,0.75,33.194649,6.672217,0.798757,0.490833,38.300559,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df_data[8:10].to_csv("../../../../Desktop/part-00000", header=None, sep='\t')

In [44]:
def sumArray(nums):
    left, right, count = {}, {}, {}
    for i, x in enumerate(nums):
        if x not in left:
            left[x] = i
        right[x] = i
        count[x] = count.get(x, 0) + 1
    ans = len(nums)
    degree = max(count.values())
    for x in count:
        if count[x]==degree:
            ans = min(ans, right[x]-left[x]+1)
    return ans
nums = [1, 2, 2, 3, 1]
sumArray(nums)

2

In [46]:
def sumArray(nums):
    left, right, count = {}, {}, {}
    for i, x in enumerate(nums):
        if x not in left:
            left[x] = i
        right[x] = i
        count[x] = count.get(x, 0) + 1
    ans = len(nums)
    degree = max(count.values())
    for x in nums:
        if count[x]==degree:
            ans = min(ans, right[x]-left[x]+1)
    return ans
            

In [47]:
def sumArray(nums):
    left, right, count = {}, {}, {}
    for i, x in enumerate(nunms):
        if x not in left:
            left[x] = i
        right[x] = i
        count[x] = count.get(x,0) + 1
    ans = len(nums)
    degree = max(len(nums))
    for x in nums:
        if count[x]==degree:
            ans = min(ans, right[x]-left[x]+1)
    return ans

In [48]:
def numSubArray(nums):
    if k<=1:
        return 0
    prod = 1
    ans = left = 0
    for right, val in enumerate(nums):
        prod *= val
        while prod>=k:
            prod/=nums[left]
            left += 1
        ans += right-left+1
    return ans

In [49]:
def numSubArray(nums):
    if k<=1:
        return 0
    prod = 1
    ans = left = 0
    for right, val in enumerate(nums):
        prod *= val
        while prod >=k:
            prod /= nums[left]
            left += 1
        ans += right-left+1
    return ans

In [50]:
def maxProfit(prices, fee):
    cash, hold = 0, -prices[0]
    for i in range(1, len(prices)):
        cash = max(cash, hold + prices[i] - fee)
        hold = max(hold, cash - prices[i])
    return cash


In [52]:
def maxProfit(prices, fee):
    cash, hold = 0, -prices[0]
    for i in range(1, len(prices)):
        cash = max(cash, hold + prices[i]-fee)
        hold = max(hold, cash-prices[i])
    return cash

In [53]:
def maxProfit(nums, fee):
    cash, hold = 0, -prices[0]
    for i in range(1, len(nums)):
        cash = max(cash, hold-nums[i]-fee)
        hold = max(hold, cash-nums[i])
    return cash

In [61]:
def maxProfit(nums, fee):
    cash, hold = 0, -nums[0]
    for item in nums[1:]:
        cash = max(cash, hold+item-fee)
        hold = max(hold, cash-item)
    return cash
prices = [1, 3, 2, 8, 4, 9]
fee = 2
maxProfit(prices, fee)

8

In [60]:
def maxProfit(prices, fee):
    cash, hold = 0, -prices[0]
    for i in range(1, len(prices)):
        cash = max(cash, hold + prices[i]-fee)
        hold = max(hold, cash-prices[i])
    return cash
prices = [1, 3, 2, 8, 4, 9]
fee = 2
maxProfit(prices, fee)

8

In [62]:
def maxProfit(nums):
    cash, hold = 0, -prices[0]
    for item in nums[1:]:
        cash = maxx(cash, hold+item-fee)
        hold = max(hold, cash-item)
    return cash

In [63]:
def isOnecahr(nums):
    i = 0
    while i<len(nums)-1:
        i += nums[i]+1
    return i==len(nums)-1

In [64]:
def isOneChar(nums):
    i = 0
    while i<len(nums)-1:
        i += nums[i]+1
    return i==len(nums)-1

In [65]:
def findLength(A, B):
    n, m = len(A), len(B)
    res = 0
    for offset in range(-m, n):
        i, m = max(offset,0), max(0, -offset)
        cur = 0
        while i<n and j<m:
            if A[i]==B[j]:
                cur += 1
                res = max(res, cur)
            else:
                cur = 0
            i += 1
            j += 1
    return res

In [66]:
def findLength(A,B):
    m, n = len(A), len(B)
    res = 0
    for offset in range(-m, n):
        i, j = max(offset, 0), max(0, -offset)
        cur = 0
        while i<n and j<n:
            if A[i]==B[j]:
                cur += 1
                res = max(cur, res)
            else:
                cur = 0
            i += 1
            j += 1
    return res

In [70]:
def findLength(A, B):
    memo = [[0] * (len(B) + 1) for _ in range(len(A) + 1)]
    print(memo)
    for i in range(len(A) - 1, -1, -1):
        for j in range(len(B) - 1, -1, -1):
            if A[i] == B[j]:
                memo[i][j] = memo[i+1][j+1]+1
    return max(max(row) for row in memo)
A=[1,2,3,2,1]
B=[3,2,1,4,7]
findLength(A,B)

[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]


3

In [72]:
def findLegn(A,B):
    memo = [[0] * (len(B) + 1) for _ in range(len(A) + 1)]
    for i in range(len(A)-1, -1, -1):
        for j in range(len(B)-1,-1,-1):
            if A[i]==B[i]:
                memo[i][j] = memo[i+1][j+1]+1
    return max(max(row) for row in memo)
    

In [73]:
def findLength(A,B):
    memo = [[0]*(len(B)+1) for _ in range(len(A)+1)]
    for i in range(len(A)-1, -1, -1):
        for j in range(len(B)-1, -1, -1):
            if A[i]==B[i]:
                memo[i][j] = memo[i+1][j+1]
    return max(max(row) for row in memo)
                   

In [74]:
def smallDis(nums, k):
    def possible(guess):
        count = left = 0
        for right, x in enumerate(nums):
            while x-nums[left]>guess:
                left += 1
            count += right-left
        return count>=k
    nums.sort()
    l = 0
    r = nums[1]-nums[0]
    while l<r:
        mid = (l+r)//2
        if possible(mid):
            r = mid
        else:
            l = mid + 1
    return l
    